
## 🔹 What is a Vector Database?
A **Vector Database** stores and retrieves **vector embeddings** (numerical representations of data like text, images, audio) and allows **similarity-based search** instead of keyword-based search.

It helps systems understand **semantic meaning**, not just exact words.

---

## 🔹 Why Do We Use a Vector Database?
Traditional databases are good for exact matches but fail at understanding meaning.

Vector DBs are used to:
- Store embeddings of documents
- Perform fast **semantic search**
- Retrieve relevant context for LLMs
- Enable **Retrieval-Augmented Generation (RAG)**

---

## 🔹 Typical Vector DB Workflow
1. Convert documents into embeddings  
2. Store embeddings in a Vector DB  
3. Convert user query into an embedding  
4. Perform similarity search (cosine / dot product / euclidean)  
5. Retrieve top-K relevant results  

---

## 🔹 Advantages of Vector Databases
- Semantic (meaning-based) search
- Very fast similarity lookups
- Scales to large datasets
- Essential for RAG pipelines
- Works across languages and domains

---

## 🔹 Disadvantages of Vector Databases
- Additional storage cost for vectors
- Re-embedding required when data changes
- Quality depends on embedding model
- Not suitable for exact queries (IDs, numbers)
- Adds system complexity

---

## 🔹 FAISS vs ChromaDB

### 🔸 FAISS (Facebook AI Similarity Search)
FAISS is a **vector search library**, not a complete database.

**Advantages:**
- Extremely fast search
- GPU acceleration support
- Highly optimized for large-scale vector search

**Disadvantages:**
- No built-in persistence
- No metadata filtering
- Requires more engineering effort

**Best Use Case:**
High-performance and large-scale similarity search systems.

---

### 🔸 ChromaDB
ChromaDB is a **full vector database** designed for LLM and RAG workflows.

**Advantages:**
- Easy to use
- Supports persistent storage
- Metadata filtering support
- Strong LangChain integration

**Disadvantages:**
- Slower than FAISS at very large scale
- Not suitable for billions of vectors

**Best Use Case:**
Local RAG systems, prototypes, and small-to-medium applications.

---

## 🔹 Other Popular Vector Databases

### 🔹 Pinecone
- Fully managed cloud-based vector database
- Highly scalable and reliable

**Pros:** No infrastructure management  
**Cons:** Paid service, requires internet access  

---

### 🔹 Weaviate
- Open-source and managed options
- Strong metadata and hybrid search support


In [23]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [24]:
loader =TextLoader('../data_source/speech.txt')
documents =loader.load()
text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=30)
docs=text_splitter.split_documents(documents)
docs

Created a chunk of size 668, which is longer than the specified 500
Created a chunk of size 982, which is longer than the specified 500
Created a chunk of size 789, which is longer than the specified 500


[Document(metadata={'source': '../data_source/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': '../data_source/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'s

In [25]:
embedding = OllamaEmbeddings(model="gemma:2b")

db = FAISS.from_documents(docs, embedding)
db

In [26]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'


## 🔹 Definition
A **Retriever** is a component in an AI / RAG system that **fetches the most relevant information** from a data source (usually a vector database) based on a user query.

It acts as a **bridge between user queries and stored knowledge**.

---

## 🔹 Why Do We Need a Retriever?
LLMs do not have access to your private or updated data.

A Retriever:
- Finds relevant documents using **semantic similarity**
- Provides accurate context to the LLM
- Reduces hallucinations
- Enables **Retrieval-Augmented Generation (RAG)**

---

## 🔹 Retriever vs Vector Database
- **Vector DB** → stores and searches embeddings  
- **Retriever** → decides *what* and *how much* to fetch from the DB  

Retriever uses the Vector DB internally.

---

## 🔹 How a Retriever Works (Step-by-Step)
1. User asks a question  
2. Query is converted into an embedding  
3. Retriever searches the vector DB  
4. Top-K relevant documents are fetched  
5. Documents are passed to the LLM  

---

## 🔹 Types of Retrievers

### 🔸 Dense Retriever
- Uses embeddings
- Semantic search

**Example:** FAISS, ChromaDB retrievers

---

### 🔸 Sparse Retriever
- Keyword-based (TF-IDF, BM25)
- Exact word matching

**Example:** Elasticsearch

---

### 🔸 Hybrid Retriever
- Combines dense + sparse
- Best accuracy

**Example:** Weaviate hybrid search

---

## 🔹 Retriever in RAG Pipeline


In [27]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

In [28]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='ecf83252-0d40-4553-8ffe-b40fb5eb6d50', metadata={'source': '../data_source/speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(2956.213)),
 (Document(id='474bb622-2c9a-41b2-a302-e2bea5b72514', metadata={'source': '../data_source/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be plante

In [29]:
db.save_local("faiss_index")

In [30]:
new_db=FAISS.load_local("faiss_index",embedding,allow_dangerous_deserialization=True)
new_db.similarity_search(query)

[Document(id='ecf83252-0d40-4553-8ffe-b40fb5eb6d50', metadata={'source': '../data_source/speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='474bb622-2c9a-41b2-a302-e2bea5b72514', metadata={'source': '../data_source/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundatio

In [31]:
docs

[Document(id='ecf83252-0d40-4553-8ffe-b40fb5eb6d50', metadata={'source': '../data_source/speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='474bb622-2c9a-41b2-a302-e2bea5b72514', metadata={'source': '../data_source/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundatio

#### ChromaDB

In [32]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [33]:
loader = TextLoader("../data_source/speech.txt")
data=loader.load()
data

[Document(metadata={'source': '../data_source/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right 

In [34]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
splits=text_splitter.split_documents(data)

In [35]:
embedding = OllamaEmbeddings(model="gemma:2b")
vectordb=Chroma.from_documents(splits,embedding=embedding)
vectordb

In [36]:
## query it
query = "What does the speaker believe is the main reason the United States should enter the war?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'